In [1]:
import zipfile
# Download zip file of pizza_steak images
# !wget https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip 

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("pizza_steak.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [28]:
import os
for dirpath, dirnames, filenames in os.walk('pizza_steak'):
    print(dirpath, dirnames, len(filenames))

    # Define training and test directory paths
train_dir = "pizza_steak/train/"
test_dir = "pizza_steak/test/"
predict_dir = "predict/"

pizza_steak ['test', 'train'] 0
pizza_steak\test ['pizza', 'steak'] 0
pizza_steak\test\pizza [] 250
pizza_steak\test\steak [] 250
pizza_steak\train ['pizza', 'steak'] 0
pizza_steak\train\pizza [] 750
pizza_steak\train\steak [] 750


In [46]:
len(os.listdir('./'))

18

In [27]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.random.set_seed(42)
# Preprocess data (get all of the pixel values between 1 and 0, also called scaling/normalization)
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Turn it into batches
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               target_size=(224, 224),
                                               class_mode='binary',
                                               batch_size=32)

test_data = test_datagen.flow_from_directory(directory=test_dir,
                                             target_size=(224, 224),
                                             class_mode='binary',
                                             batch_size=32)



Found 1500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Found 56679 images belonging to 15 classes.


In [41]:
predict_datagen = ImageDataGenerator(rescale=1./255)
predict_data = predict_datagen.flow_from_directory(directory=predict_dir,
                                             target_size=(224, 224),
                                             class_mode='binary',
                                             batch_size=32)

Found 5 images belonging to 2 classes.


In [47]:
len(test_data)
for a, b in test_data:
    print(a,b)
    break

[[[[0.19607845 0.12156864 0.09803922]
   [0.19607845 0.1137255  0.10196079]
   [0.18039216 0.09803922 0.09411766]
   ...
   [0.14901961 0.07058824 0.1137255 ]
   [0.15294118 0.08627451 0.1254902 ]
   [0.16470589 0.09803922 0.13725491]]

  [[0.20000002 0.11764707 0.09803922]
   [0.20000002 0.11764707 0.09803922]
   [0.20000002 0.10588236 0.10588236]
   ...
   [0.17254902 0.09803922 0.12156864]
   [0.1764706  0.10980393 0.14117648]
   [0.18431373 0.11764707 0.14901961]]

  [[0.20784315 0.11764707 0.09411766]
   [0.20784315 0.1137255  0.09803922]
   [0.20000002 0.10588236 0.09803922]
   ...
   [0.16470589 0.09019608 0.10588236]
   [0.15294118 0.09019608 0.10196079]
   [0.15294118 0.09019608 0.10196079]]

  ...

  [[0.49411768 0.3019608  0.14117648]
   [0.5019608  0.30980393 0.14901961]
   [0.49803925 0.30588236 0.14509805]
   ...
   [0.5176471  0.31764707 0.14901961]
   [0.50980395 0.30588236 0.14901961]
   [0.5058824  0.3019608  0.14901961]]

  [[0.49411768 0.3019608  0.14117648]
   [0.4

In [6]:
# Make the creating of our model a little easier
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Activation
from tensorflow.keras import Sequential

# Create the model (this can be our baseline, a 3 layer Convolutional Neural Network)
model_4 = Sequential([
  Conv2D(filters=10, 
         kernel_size=3, 
         strides=1,
         padding='valid',
         activation='relu', 
         input_shape=(224, 224, 3)), # input layer (specify input shape). default color_mode used above is rgb - hence 3
  Conv2D(10, 3, activation='relu'),
  Conv2D(10, 3, activation='relu'),
  Flatten(),
  Dense(1, activation='sigmoid') # output layer (specify output shape)
])

# Compile the model
model_4.compile(loss='binary_crossentropy',
                optimizer=Adam(),
                metrics=['accuracy'])

In [7]:
# Fit the model
history_4 = model_4.fit(train_data,
                        epochs=3,
                        steps_per_epoch=len(train_data),
                        validation_data=test_data,
                        validation_steps=len(test_data))

Epoch 1/3
47/47 [==============================] - 57s 1s/step - loss: 0.8966 - accuracy: 0.6473 - val_loss: 0.5731 - val_accuracy: 0.7140
Epoch 2/3
47/47 [==============================] - 55s 1s/step - loss: 0.4508 - accuracy: 0.8013 - val_loss: 0.4179 - val_accuracy: 0.8200
Epoch 3/3
47/47 [==============================] - 51s 1s/step - loss: 0.3864 - accuracy: 0.8407 - val_loss: 0.3914 - val_accuracy: 0.8200


In [8]:
model_4.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 10)      280       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 10)      910       
                                                                 
 conv2d_2 (Conv2D)           (None, 218, 218, 10)      910       
                                                                 
 flatten (Flatten)           (None, 475240)            0         
                                                                 
 dense (Dense)               (None, 1)                 475241    
                                                                 
Total params: 477,341
Trainable params: 477,341
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Evaluate on the test data
model_4.evaluate(test_data)

16/16 [==============================] - 5s 278ms/step - loss: 0.3914 - accuracy: 0.8200


[0.39142265915870667, 0.8199999928474426]

In [10]:
# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224):
  """
  Reads an image from filename, turns it into a tensor
  and reshapes it to (img_shape, img_shape, colour_channel).
  """
  # Read in target file (an image)
  img = tf.io.read_file(filename)

  # Decode the read file into a tensor & ensure 3 colour channels 
  # (our model is trained on images with 3 colour channels and sometimes images have 4 colour channels)
  img = tf.image.decode_image(img, channels=3)

  # Resize the image (to the same size our model was trained on)
  img = tf.image.resize(img, size = [img_shape, img_shape])

  # Rescale the image (get all values between 0 and 1)
  img = img/255.
  return img

In [42]:
# Load in and preprocess our custom image
myimg = load_and_prep_image("steak.jpg")
model_4.predict(tf.expand_dims(myimg, axis=0))

model_4.predict(predict_data)


array([[0.28513056],
       [0.2973953 ],
       [0.31302595],
       [0.13253406],
       [0.02717996]], dtype=float32)

In [43]:
# mypizza.shape, tf.expand_dims(mypizza, axis=0)